# Data loading

In [ ]:
# from fraudetect.dataset import load_data


# Feature engineering

In [ ]:
from fraudetect import import_from_path, sample_cfg

configs = import_from_path(
    "hyp_search_conf", r"D:\fraud-detection-galsen\tools\hyp_search_conf.py"
)

In [ ]:
from fraudetect.features import (
    fit_outliers_detectors,
    concat_outliers_probs_pyod,
    load_transforms_pyod,
)
from fraudetect.detectors import get_detector, instantiate_detector

model_list = list()
# names = configs.outliers_detectors.keys()
names = ["cblof", "iforest"]

outliers_det_configs = []
for name in sorted(names):
    detector, cfg = get_detector(name=name, config=configs.outliers_detectors)
    cfg = sample_cfg(cfg)
    detector = instantiate_detector(detector, cfg)
    model_list.append(detector)

    cfg["detector"] = configs.outliers_detectors[name]["detector"]
    outliers_det_configs.append((name, cfg))

model_list = fit_outliers_detectors(model_list, X_train)
X_t = concat_outliers_probs_pyod(
    fitted_detector_list=model_list,
    X=X_train,
    method="linear",
    add_confidence=True,
)

outliers_det_configs = OrderedDict(outliers_det_configs)
transform = load_transforms_pyod(
    X_train=X_train,
    outliers_det_configs=outliers_det_configs,
    method="linear",
    add_confidence=False,
)